In [1]:
!pip install ultralytics optuna pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 13.2 MB/s eta 0:00:00a 0:00:01


In [2]:
from ultralytics import YOLO
import optuna
import os
import yaml

In [3]:
# Set this environment variable to disable strict metric checking
os.environ['TUNE_DISABLE_STRICT_METRIC_CHECKING'] = '1'

In [4]:
def train_yolov8(trial):
    model = YOLO("yolov8x.pt")  # Load a pretrained YOLOv8 model

    # Set up the dataset path
    data_path = "/kaggle/input/vehicle-detection/data.yaml"  # Replace with your dataset path
    print(f"Training on dataset: {data_path}")

    # Suggest hyperparameters
    config = {
        "epochs": trial.suggest_int("epochs", 5, 10),
        "batch_size": trial.suggest_categorical("batch_size", [8]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),  # Correct loguniform usage
        "image_size": trial.suggest_categorical("image_size", [720])
    }

    print(f"Starting training with config: {config}")

    results = model.train(
        data=data_path,
        epochs=config["epochs"],
        batch=config["batch_size"],
        lr0=config["learning_rate"],
        imgsz=config["image_size"]
    )

    print("Training completed, extracting metrics...")

    # Extract validation metrics
    metrics = results.results_dict
    print(f"Validation metrics: {metrics}")

    # Print results per class
    with open(data_path, 'r') as f:
        data_dict = yaml.safe_load(f)
    class_names = data_dict['names']

    # Return the metric to be optimized
    return metrics['metrics/mAP50-95(B)']

In [5]:
def objective(trial):
    return train_yolov8(trial)

In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2024-06-25 07:33:27,106] A new study created in memory with name: no-name-401c21ac-1523-49ed-a81a-b8ba61462182


100%|██████████| 131M/131M [00:00<00:00, 255MB/s]  


Training on dataset: /kaggle/input/vehicle-detection/data.yaml
Starting training with config: {'epochs': 9, 'batch_size': 8, 'learning_rate': 1.1792311166139625e-05, 'image_size': 720}
Ultralytics YOLOv8.2.42 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/vehicle-detection/data.yaml, epochs=9, time=None, patience=100, batch=8, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fals

/tmp/ipykernel_34/3428001106.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),  # Correct loguniform usage


100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]
2024-06-25 07:33:30,739	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-25 07:33:32,070	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:13<00:00,  1.02s/it]


                   all        200       1199      0.598      0.502      0.561      0.292
                   car        159        743      0.642      0.881      0.861      0.411
                  bike         99        125      0.427      0.736      0.722      0.436
                  auto         47         60      0.502      0.369      0.473      0.218
                 cycle         29         46      0.338        0.1      0.143     0.0783
                   bus         48         52      0.846      0.741      0.792      0.393
             minitruck         54         58      0.529      0.362      0.348      0.154
                 truck         32         47       0.69      0.532      0.609      0.306
                   van         25         28      0.267      0.214      0.167     0.0971
                  taxi         10         10          1      0.984      0.995       0.62
                  toto         28         30       0.74        0.1        0.5      0.211
Speed: 0.3ms preproce

lr/pg0,▃▆█▇▅▄▂▁
lr/pg1,▃▆█▇▅▄▂▁
lr/pg2,▃▆█▇▅▄▂▁
metrics/mAP50(B),▄▁▄▄▆▇██
metrics/mAP50-95(B),▃▁▃▄▅▆██
metrics/precision(B),▆▁▃▂▇█▆▆
metrics/recall(B),▄▁▄▆▆▅██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▆▅▄▂▁


[I 2024-06-25 11:33:20,643] Trial 4 finished with value: 0.292357398704865 and parameters: {'epochs': 8, 'batch_size': 8, 'learning_rate': 2.0404858590143924e-05, 'image_size': 720}. Best is trial 3 with value: 0.292357398704865.


Training completed, extracting metrics...
Validation metrics: {'metrics/precision(B)': 0.5981484003645128, 'metrics/recall(B)': 0.5019603525268939, 'metrics/mAP50(B)': 0.5608312550982388, 'metrics/mAP50-95(B)': 0.292357398704865, 'fitness': 0.3192047843442024}


In [7]:
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value: 0.292357398704865
  Params: 
    epochs: 8
    batch_size: 8
    learning_rate: 2.645620762943047e-05
    image_size: 720


In [8]:
# Train the model with the best hyperparameters
best_model = YOLO("yolov8x.pt")
best_results = best_model.train(
        data="/kaggle/input/vehicle-detection/data.yaml",
        epochs=trial.params["epochs"],
        batch=trial.params["batch_size"],
        lr0=trial.params["learning_rate"],
        imgsz=trial.params["image_size"]
)

Ultralytics YOLOv8.2.42 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/vehicle-detection/data.yaml, epochs=8, time=None, patience=100, batch=8, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
WARNING ⚠️ imgsz=[720] must be multiple of max stride 32, updating to [736]


train: Scanning /kaggle/input/vehicle-detection/train/labels... 2600 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:02<00:00, 989.26it/s] 

train: WARNING ⚠️ /kaggle/input/vehicle-detection/train/images/rainy day (236).jpg: 1 duplicate labels removed


train: WARNING ⚠️ Cache directory /kaggle/input/vehicle-detection/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        200       1199      0.624      0.466      0.564      0.279



8 epochs completed in 0.760 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 136.7MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.2.42 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 268 layers, 68138013 parameters, 0 gradients, 257.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:13<00:00,  1.01s/it]


                   all        200       1199      0.598      0.502      0.561      0.292
                   car        159        743      0.642      0.881      0.861      0.411
                  bike         99        125      0.427      0.736      0.722      0.436
                  auto         47         60      0.502      0.369      0.473      0.218
                 cycle         29         46      0.338        0.1      0.143     0.0783
                   bus         48         52      0.846      0.741      0.792      0.393
             minitruck         54         58      0.529      0.362      0.348      0.154
                 truck         32         47       0.69      0.532      0.609      0.306
                   van         25         28      0.267      0.214      0.167     0.0971
                  taxi         10         10          1      0.984      0.995       0.62
                  toto         28         30       0.74        0.1        0.5      0.211
Speed: 0.3ms preproce

lr/pg0,▃▆█▇▅▄▂▁
lr/pg1,▃▆█▇▅▄▂▁
lr/pg2,▃▆█▇▅▄▂▁
metrics/mAP50(B),▄▁▄▄▆▇██
metrics/mAP50-95(B),▃▁▃▄▅▆██
metrics/precision(B),▆▁▃▂▇█▆▆
metrics/recall(B),▄▁▄▆▆▅██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▆▅▄▂▁


In [9]:
print("\nFinal results with best hyperparameters:")
print(best_results)


Final results with best hyperparameters:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d82f881cd00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.0